In [39]:
import os
import pandas as pd
from pandas import read_csv, read_excel, DataFrame
from sae import COTSCar
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum as qsum
import numpy as np
from typing import List

import sys
sys.path.append('../')
import saedfsc

Price of the car.

In [40]:
maxProductPrice = 120000

Information about customers

In [41]:
customers = saedfsc.customers['Name'].to_list()
cQty = dict(zip(saedfsc.customers['Name'], saedfsc.customers['Quantity'])) # customer quantities
cPriceFocus = dict(zip(saedfsc.customers['Name'], saedfsc.customers['PriceFocus']))
name_weights_dict = saedfsc.customers.set_index('Name')['PerformanceUtilityWeights'].to_dict()
cWts = {c : np.fromstring(name_weights_dict[c].strip('[]'), sep=',') for c in name_weights_dict}

Methods to help compute the market share for each customer type (based on logit model).

In [42]:
def getTotalUtilityForCustomer(car : COTSCar, c : str):
    total_utility = 0
    perf_utility = car.partworth_objectives(weights=cWts[c])[0]
    total_utility += (1-cPriceFocus[c])*perf_utility - cPriceFocus[c]*pricePerf
    return total_utility

def getMarketShare(car : COTSCar, c : str, competitors : List[COTSCar]): # based on logit model of demand
    carUtility = getTotalUtilityForCustomer(car, c)
    totalCompetitorUtility = sum([getTotalUtilityForCustomer(competitorCar, c) for competitorCar in competitors])
    return carUtility / (totalCompetitorUtility + carUtility)

Create a set of competitors.

In [43]:
profitMargin = 0.1
competitors = []
for i in range(3):
    competitorCar = COTSCar()
    competitorCar.price = (1+profitMargin)*competitorCar.cost()
    competitorCar.name = "Competitor Car " + str(i)
    competitors.append(competitorCar)

names = [car.name for car in competitors]
designs = [car.vector for car in competitors]
salesPrices = [car.price for car in competitors]
costs = [car.cost() for car in competitors]
competitorsDF = pd.DataFrame({
    'Name': names,
    'Design': designs,
    'SalesPrice': salesPrices,
    'ProductCost': costs
})
competitorsDF

,Name,Design,SalesPrice,ProductCost
0,Competitor Car 0,"[9, 8, 9, 9, 7, 1, 5, 3, 13, 0, 16, 93, 2, 0, ...",289015.315226,262741.195660
1,Competitor Car 1,"[3, 8, 8, 8, 1, 1, 2, 18, 10, 4, 48, 119, 1, 0...",87114.519910,79195.018100
2,Competitor Car 2,"[7, 2, 9, 10, 4, 6, 3, 5, 9, 0, 6, 169, 4, 2, ...",90732.487155,82484.079232


Create a random car and compute market share

In [47]:
carDesignVec = [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
product = COTSCar()
product.vector = carDesignVec
product.price = (1+profitMargin)*product.cost()
print("OUR CAR:") 
print("Design: ", product.vector)
print("Sales price:", product.price)

utilities = [getTotalUtilityForCustomer(product, c) for c in customers]
market_shares = [getMarketShare(product, c, competitors) for c in customers]
expected_customer_demand = [getMarketShare(product, c, competitors)*cQty[c] for c in customers]

df = pd.DataFrame({
    'Name': customers,
    'Utility for our car': utilities,
    'Market share': market_shares,
    'Expected demand': expected_customer_demand
})

#market_shares_df = pd.DataFrame(list(market_shares.items()), columns=['Customer Type', 'MarketShare'])
#customer_demand_df = pd.DataFrame(list(expected_customer_demand.items()), columns=['Customer Type', 'Expected Demand'])
#df = pd.merge(market_shares_df, customer_demand_df, on='Customer Type')
df

OUR CAR:
Design:  [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
Sales price: 226723.930004


,Name,Utility for our car,Market share,Expected demand
0,CustomerType1,-0.288524,0.301743,62.460866
1,CustomerType2,-1.751465,0.494041,178.842765
2,CustomerType3,-2.381956,0.565429,31.663997
3,CustomerType4,-0.583812,0.373786,105.407536
4,CustomerType5,-1.209188,0.529700,329.473203
5,CustomerType6,-0.766024,0.476764,173.065344
6,CustomerType7,-2.407024,0.567116,558.609348
7,CustomerType8,-1.100016,0.486611,372.743709
8,CustomerType9,-2.357987,0.577319,66.969003
9,CustomerType10,-0.766914,0.352264,268.777771
